In [ ]:
import os, numpy as np
from SQDMetal.COMSOL.Model import COMSOL_Model
from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
from adjoint_sim_sf import Experiment
from datetime import datetime
from adjoint_sim_sf import AdjointEvaluator, Optimiser

if COMSOL_Model._engine is None:
    COMSOL_Model.init_engine()

# 2) Config (only keys that AdjointEvaluator.update_params understands)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 20,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

# 3) Objects
designer   = SymmetricTransmonDesign()
evaluator  = AdjointEvaluator(designer, config=base_config)
experiment = Experiment(name=f"grid_{datetime.now():%Y%m%d_%H%M%S}")

# 4) 3x3 grid
x_vals = np.array([0.19, 0.20, 0.21], float)
y_vals = np.array([0.23, 0.25, 0.27], float)
param_grid = np.array([[x, y] for x in x_vals for y in y_vals], dtype=float)

# 5) Optimiser + sweep
initial_params = np.array([0.20, 0.20], float)
lr = 0.01
opt = Optimiser(initial_params=initial_params, lr=lr, evaluator=evaluator)

results = opt.sweep(param_grid, perturbation_mag=None, verbose=True)

# 6) Persist config + results
# Save the exact evaluator config that actually ran (arrays converted to lists)
experiment.save_config({
    "adjoint_evaluator": evaluator.to_config_dict(),
    "grid": {"x_vals": x_vals.tolist(), "y_vals": y_vals.tolist()},
    "initial_params": initial_params.tolist(),
    "lr": lr,
})

results_filename = f"sweep_{datetime.now():%Y%m%d_%H%M%S}.jsonl"
experiment.save_results(results, results_filename)

print("Wrote:", experiment.path(results_filename))


Params: [0.19 0.23],Loss: -53958274241889.875, ||grad||: 5.759800e+17, grad: [-4.15396668e+17 -3.98996936e+17]
Params: [0.19 0.25],Loss: -3242125454566068.5, ||grad||: 4.392382e+19, grad: [-3.13396035e+19 -3.07754985e+19]
Params: [0.19 0.27],Loss: -6806547480579875.0, ||grad||: 1.758316e+19, grad: [1.51288115e+19 8.96028266e+18]
Params: [0.2  0.23],Loss: -265927330460684.75, ||grad||: 2.227439e+18, grad: [-1.57788532e+18 -1.57218426e+18]
Params: [0.2  0.25],Loss: -9407775705758358.0, ||grad||: 1.973341e+20, grad: [-1.41439169e+20 -1.37607068e+20]
Params: [0.2  0.27],Loss: -3562583939890480.5, ||grad||: 2.120473e+18, grad: [1.81434602e+18 1.09752256e+18]
Params: [0.21 0.23],Loss: -1900199063072076.0, ||grad||: 2.406089e+19, grad: [-1.95932371e+19 -1.39653590e+19]
Params: [0.21 0.25],Loss: -1.0532288533235558e+16, ||grad||: 8.635853e+19, grad: [7.40044281e+19 4.45100029e+19]
Params: [0.21 0.27],Loss: -2487345338543664.5, ||grad||: 5.608902e+17, grad: [5.17001673e+17 2.17502042e+17]
Wrote